In [54]:
import altair as alt
import pandas as pd
import numpy as np

## Rabbit vs HTTP

In [55]:
results = pd.read_csv('../Http_vs_Rabbit/report.csv', delimiter=';')

# Assuming the units are in the format "1234 unit", we split on space and take the first part
results['Mean'] = results['Mean'].str.split(' ').str[0]
results['Error'] = results['Error'].str.split(' ').str[0]
results['Median'] = results['Median'].str.split(' ').str[0]
results['Min'] = results['Min'].str.split(' ').str[0]
results['Max'] = results['Max'].str.split(' ').str[0]
results['StdDev'] = results['StdDev'].str.split(' ').str[0]

results

,Method,Job,AnalyzeLaunchVariance,EvaluateOverhead,MaxAbsoluteError,MaxRelativeError,MinInvokeCount,MinIterationTime,OutlierMode,Affinity,...,WarmupCount,MessageSize,Mean,Error,StdDev,Min,Max,Median,Ratio,RatioSD
0,HTTP,DefaultJob,False,Default,Default,Default,Default,Default,Default,11111111111111111111,...,Default,1024,4.562,0.0908,0.0849,4.417,4.705,4.577,1.00,0.00
1,Rabbit,DefaultJob,False,Default,Default,Default,Default,Default,Default,11111111111111111111,...,Default,1024,4.459,0.0865,0.1030,4.273,4.617,4.484,0.97,0.03
2,Rabbit-RT,DefaultJob,False,Default,Default,Default,Default,Default,Default,11111111111111111111,...,Default,1024,497.748,5.6553,5.0133,480.357,499.472,499.147,109.19,2.64
3,HTTP,DefaultJob,False,Default,Default,Default,Default,Default,Default,11111111111111111111,...,Default,102400,27.692,1.9728,5.8168,16.060,40.116,27.871,1.00,0.00
4,Rabbit,DefaultJob,False,Default,Default,Default,Default,Default,Default,11111111111111111111,...,Default,102400,9.341,0.3256,0.9499,7.607,11.703,9.178,0.35,0.10
5,Rabbit-RT,DefaultJob,False,Default,Default,Default,Default,Default,Default,11111111111111111111,...,Default,102400,13.222,0.2617,0.5800,12.058,14.683,13.177,0.51,0.11
6,HTTP,DefaultJob,False,Default,Default,Default,Default,Default,Default,11111111111111111111,...,Default,524288,63.652,2.7262,7.8657,47.678,82.814,63.766,1.00,0.00
7,Rabbit,DefaultJob,False,Default,Default,Default,Default,Default,Default,11111111111111111111,...,Default,524288,42.717,2.2626,6.6713,29.521,58.846,42.714,0.68,0.13
8,Rabbit-RT,DefaultJob,False,Default,Default,Default,Default,Default,Default,11111111111111111111,...,Default,524288,49.532,0.9608,1.3469,46.138,51.998,49.858,0.82,0.10
9,HTTP,DefaultJob,False,Default,Default,Default,Default,Default,Default,11111111111111111111,...,Default,1048576,109.564,4.2685,12.4513,86.581,145.882,106.839,1.00,0.00


In [56]:
base = alt.Chart(results).transform_calculate(
    MessageSizeKB='datum.MessageSize / 1024'  # Convert bytes to kilobytes
).properties(
    width=400
)

In [57]:
messageSizes = (results['MessageSize'].unique())
methods = results['Method'].unique()


In [58]:
def create_chart_for_msg_size(messageSize):
    return base.mark_bar().transform_filter(
        alt.datum.MessageSize == messageSize
    ).encode(
        x=alt.X('Mean:Q', title='Mean (\u03BCs)'),
        y=alt.Y('MessageSize:O', title=None).axis(format='.0s'),
        color=alt.Color('Method:N',title=None, legend=None),
        row=alt.Row('Method:N', title=None),
    )


charts = [create_chart_for_msg_size(messageSize) for messageSize in messageSizes]
alt.vconcat(*charts, center=True).resolve_scale(x='shared').resolve_axis(x='shared', y='shared')

alt.VConcatChart(...)

## MassTransit vs RabbitMQ Client


In [59]:
m_results = pd.read_csv('../MassTransitRabbit_vs_RabbitClient/report.csv', delimiter=';')
m_results['MeanNs'] = m_results['Mean'] * 1000

In [60]:
messageSizes = (m_results['MessageSize'].unique())
methods = m_results['Method'].unique()

In [86]:
m_base = alt.Chart(m_results).transform_calculate(
    MessageSizeKB='datum.MessageSize / 1024',# Convert bytes to kilobytes
    MeanNs='datum.Mean / 1000'
).properties(
    width=400
)

In [93]:
def create_chart_for_msg_size(messageSize):
    return m_base.mark_bar().transform_filter(
        alt.datum.MessageSize == messageSize
    ).encode(
        x=alt.X('MeanNs:Q', title='Mean (ns)'),
        y=alt.Y('MessageSize:O', title=None).axis(format='.0s'),
        color=alt.Color('Method:N',title=None, legend=None),
        row=alt.Row('Method:O', title=None,)
    )

charts = [create_chart_for_msg_size(messageSize) for messageSize in messageSizes]
alt.vconcat(*charts, center=True).resolve_scale(x='shared').resolve_axis(x='shared', y='shared')

alt.VConcatChart(...)